# Caipora Project

__Objetivo__




 
__Data Source__

https://queimadas.dgi.inpe.br/queimadas/portal

https://ipsamazonia.org.br/

https://openaq.org/


__Data characteristics__

- Time Series;
- Geographic coordinates – Latitude/Longitude;
- Satellite Name

In [1]:
import os
import math
import unicodedata

import glob

import datetime
import gmaps

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

## Prerequisite

In [2]:
WORKDIR = os.path.abspath(os.getcwd())

## Get the data

### Hotspot data

__List files used in the analysis__

In [3]:
path = ''.join([WORKDIR, "/data/states/*"]) 
hotspot_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [4]:
hotspot_df = pd.concat(map(pd.read_csv, hotspot_files))
hotspot_df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,CURURUPU,Amazonia,0.0,0.8,0.0,-1.87136,-44.78587,NaN
1,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,MARACACUME,Amazonia,0.0,0.1,0.1,-1.82566,-45.88670,NaN
2,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,BURITICUPU,Amazonia,0.0,1.1,0.1,-4.57874,-46.38660,NaN
3,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,PAULO RAMOS,Amazonia,0.0,1.4,0.1,-4.59554,-45.66039,NaN
4,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,ARAME,Amazonia,0.0,0.4,0.3,-5.21960,-46.12886,NaN


In [5]:
hotspot_df.estado.unique()

array(['MARANHAO', 'MATO GROSSO', 'ACRE', 'TOCANTINS', 'AMAZONAS',
       'RORAIMA', 'AMAPA', 'RONDONIA', 'PARA'], dtype=object)

### Amazon SPI data

__List files used in the analysis__

In [6]:
path = ''.join([WORKDIR, "/data/spi/amazonia/compactado"]) 
spi_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [7]:
spi_df = pd.concat(map(pd.read_csv, spi_files))
spi_df.head()

,cibge,municipio,estado,ano,ips,rankips,nhbs,fbes,opts
0,1100015.0,ALTA FLORESTA D'OESTE,RONDONIA,2014.0,56.594170,59.0,60.195511,57.369340,52.217659
1,1100023.0,ARIQUEMES,RONDONIA,2014.0,55.728511,73.0,59.059533,59.304304,48.821695
2,1100031.0,CABIXI,RONDONIA,2014.0,58.915724,92.0,74.178132,54.787265,47.781775
3,1100049.0,CACOAL,RONDONIA,2014.0,61.717654,72.0,72.691093,60.613765,51.848105
4,1100056.0,CEREJEIRAS,RONDONIA,2014.0,54.593926,59.0,62.606094,62.512401,38.663282


## Explore the data

### Describe the data I

__Hotspot__

In [8]:
hotspot_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
diasemchuva,11060285.0,7.623764,70.880709,-999.00000,1.00000,4.00000,12.000,120.00000
precipitacao,11060285.0,1.034430,3.854016,0.00000,0.00000,0.00000,0.300,203.70000
riscofogo,11060285.0,-5.310438,77.318383,-999.00000,0.40000,0.90000,1.000,1.00000
latitude,12651992.0,-7.019591,3.980039,-16.29000,-9.85041,-7.71500,-3.879,5.23000
longitude,12651992.0,-56.207436,6.320051,-73.93146,-61.00200,-55.22246,-51.210,-43.64537
frp,3390008.0,19.308732,59.705760,-3.70000,3.00000,7.20000,16.800,9722.60000


__Amazon SPI__

In [9]:
spi_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
cibge,2316.0,2.288414e+06,1.365403e+06,1.100015e+06,1.502798e+06,1.713254e+06,2.109278e+06,5.108956e+06
ano,2316.0,2.017667e+03,2.868061e+00,2.014000e+03,2.014000e+03,2.018000e+03,2.021000e+03,2.021000e+03
ips,2316.0,5.456015e+01,3.811311e+00,4.359790e+01,5.192879e+01,5.430994e+01,5.680819e+01,7.441633e+01
rankips,2316.0,4.833290e+01,2.886438e+01,0.000000e+00,2.300000e+01,4.700000e+01,7.300000e+01,9.900000e+01
nhbs,2319.0,6.437410e+01,6.825254e+00,4.337849e+01,5.969696e+01,6.432076e+01,6.887852e+01,8.712833e+01
fbes,2319.0,5.499606e+01,5.495901e+00,3.929388e+01,5.094986e+01,5.470515e+01,5.837245e+01,7.870139e+01
opts,2319.0,4.429596e+01,6.039469e+00,2.126966e+01,4.106181e+01,4.519378e+01,4.836210e+01,7.014162e+01


### Get information about data

__Hotspot__

In [10]:
hotspot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12651992 entries, 0 to 318737
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   datahora      object 
 1   satelite      object 
 2   pais          object 
 3   estado        object 
 4   municipio     object 
 5   bioma         object 
 6   diasemchuva   float64
 7   precipitacao  float64
 8   riscofogo     float64
 9   latitude      float64
 10  longitude     float64
 11  frp           float64
dtypes: float64(6), object(6)
memory usage: 1.2+ GB


__Amazon SPI__

In [11]:
spi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2319 entries, 0 to 772
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   cibge      2316 non-null   float64
 1   municipio  2316 non-null   object 
 2   estado     2316 non-null   object 
 3   ano        2316 non-null   float64
 4   ips        2316 non-null   float64
 5   rankips    2316 non-null   float64
 6   nhbs       2319 non-null   float64
 7   fbes       2319 non-null   float64
 8   opts       2319 non-null   float64
dtypes: float64(7), object(2)
memory usage: 181.2+ KB


### Percentage of missing data

In [12]:
def get_pct_missing_data(dataset):
    """
    Get Percentage of missing data
    
    Attributes
    ----------
    dataset : Pandas DataFrame
    """
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum() / dataset.isnull().count() * 100
    percent = (round(percent, 4)).sort_values(ascending=False)

    missing_data = pd.concat([total, percent], keys=["Total", '%'], axis=1)

    return missing_data

__Hotspot__

In [13]:
get_pct_missing_data(hotspot_df)

,Total,%
frp,9261984,73.2057
diasemchuva,1591707,12.5807
precipitacao,1591707,12.5807
riscofogo,1591707,12.5807
datahora,0,0.0000
satelite,0,0.0000
pais,0,0.0000
estado,0,0.0000
municipio,0,0.0000
bioma,0,0.0000


__Amazon SPI__

In [14]:
get_pct_missing_data(spi_df)

,Total,%
cibge,3,0.1294
municipio,3,0.1294
estado,3,0.1294
ano,3,0.1294
ips,3,0.1294
rankips,3,0.1294
nhbs,0,0.0000
fbes,0,0.0000
opts,0,0.0000


## Prepare the data

### Fix or remove outliers (optional)

__Hotspot__

In [15]:
hotspot_df.loc[hotspot_df["riscofogo"] < 0, "riscofogo"] = 0
hotspot_df.loc[hotspot_df["diasemchuva"] < 0, "diasemchuva"] = 0

__Amazon SPI__

In [16]:
spi_df.dropna(subset=["cibge", "municipio", "estado", "ips"], inplace=True)

In [17]:
def purge_spec_chars(word):
    """
    Remove all special characters in the word.
    
    Attributes
    ----------
    word : str
    
    Returns
    -------
    word_without_spec_chars: str
    """
    nfkd_form = unicodedata.normalize("NFKD", word)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [18]:
spi_df.municipio = spi_df.municipio.apply(purge_spec_chars)

### Fill in missing values (e.g., with zero, mean, median...) or drop their rows (or columns)

__Hotspot__

In [19]:
hotspot_df['frp'].fillna(value=0, inplace=True)
hotspot_df['riscofogo'].fillna(value=0, inplace=True)
hotspot_df['diasemchuva'].fillna(value=0, inplace=True)
hotspot_df['precipitacao'].fillna(value=0, inplace=True)

### Parse datehour of string to date

__Hotspot__

In [20]:
hotspot_df['datahora'] = pd.to_datetime(hotspot_df['datahora'], format='%Y/%m/%d %H:%M:%S')

### Add Year and Month columns

__Hotspot__

In [21]:
hotspot_df["ano"] = pd.DatetimeIndex(hotspot_df['datahora']).year
hotspot_df["mes"] = pd.DatetimeIndex(hotspot_df['datahora']).month

### Describe the data II

__Hotspot__

In [22]:
hotspot_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
diasemchuva,12651992.0,10.617849,19.514854,0.00000,0.00000,3.00000,10.000,120.00000
precipitacao,12651992.0,0.904291,3.619731,0.00000,0.00000,0.00000,0.200,203.70000
riscofogo,12651992.0,0.613064,0.401937,0.00000,0.20000,0.80000,1.000,1.00000
latitude,12651992.0,-7.019591,3.980039,-16.29000,-9.85041,-7.71500,-3.879,5.23000
longitude,12651992.0,-56.207436,6.320051,-73.93146,-61.00200,-55.22246,-51.210,-43.64537
frp,12651992.0,5.173632,32.066891,-3.70000,0.00000,0.00000,1.100,9722.60000
ano,12651992.0,2017.520379,2.792640,2012.00000,2015.00000,2018.00000,2020.000,2022.00000
mes,12651992.0,8.714163,1.990419,1.00000,8.00000,9.00000,10.000,12.00000


In [23]:
get_pct_missing_data(hotspot_df)

,Total,%
datahora,0,0.0
satelite,0,0.0
pais,0,0.0
estado,0,0.0
municipio,0,0.0
bioma,0,0.0
diasemchuva,0,0.0
precipitacao,0,0.0
riscofogo,0,0.0
latitude,0,0.0


__Amazon SPI__

In [24]:
spi_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
cibge,2316.0,2.288414e+06,1.365403e+06,1.100015e+06,1.502798e+06,1.713254e+06,2.109278e+06,5.108956e+06
ano,2316.0,2.017667e+03,2.868061e+00,2.014000e+03,2.014000e+03,2.018000e+03,2.021000e+03,2.021000e+03
ips,2316.0,5.456015e+01,3.811311e+00,4.359790e+01,5.192879e+01,5.430994e+01,5.680819e+01,7.441633e+01
rankips,2316.0,4.833290e+01,2.886438e+01,0.000000e+00,2.300000e+01,4.700000e+01,7.300000e+01,9.900000e+01
nhbs,2316.0,6.438251e+01,6.825672e+00,4.337849e+01,5.972034e+01,6.432380e+01,6.888298e+01,8.712833e+01
fbes,2316.0,5.498789e+01,5.494759e+00,3.929388e+01,5.094770e+01,5.470459e+01,5.836069e+01,7.870139e+01
opts,2316.0,4.431006e+01,6.030640e+00,2.126966e+01,4.108251e+01,4.519826e+01,4.836412e+01,7.014162e+01


In [25]:
get_pct_missing_data(spi_df)

,Total,%
cibge,0,0.0
municipio,0,0.0
estado,0,0.0
ano,0,0.0
ips,0,0.0
rankips,0,0.0
nhbs,0,0.0
fbes,0,0.0
opts,0,0.0
